**Dependencies**

In [1]:
import os
from tokens import token as _token
from download_functions import sync

**Run this line to download files**

In [2]:
# set parameters for download
token = _token
url = "https://ladsweb.modaps.eosdis.nasa.gov/archive/orders/501444664/"
destination = "raw/order/"

# create destination storage directory
try:
    os.mkdir(destination)
except:
    print ("Directory already exists.")

Directory already exists.


In [3]:
# download fils
sync(url, destination, token)

skipping:  raw/order/MOD04_3K.A2015365.1745.061.2017324180501.psgscs_000501444664.hdf
skipping:  raw/order/MOD04_3K.A2016001.1650.061.2017324183141.psgscs_000501444664.hdf
skipping:  raw/order/MOD04_3K.A2016002.1735.061.2017324183949.psgscs_000501444664.hdf
skipping:  raw/order/MOD04_3K.A2016003.mosaic.061.2020135184456.psmcgscs_000501444664.Optical_Depth_Land_And_Ocean.hdf
skipping:  raw/order/MOD04_3K.A2016004.1720.061.2017324192948.psgscs_000501444664.hdf
skipping:  raw/order/MOD04_3K.A2016005.1805.061.2017324195444.psgscs_000501444664.hdf
skipping:  raw/order/MOD04_3K.A2016006.1710.061.2017324225921.psgscs_000501444664.hdf
skipping:  raw/order/MOD04_3K.A2016007.1750.061.2017324225643.psgscs_000501444664.hdf
skipping:  raw/order/MOD04_3K.A2016008.1655.061.2017324230556.psgscs_000501444664.hdf
skipping:  raw/order/MOD04_3K.A2016009.1740.061.2017324230916.psgscs_000501444664.hdf
skipping:  raw/order/MOD04_3K.A2016010.mosaic.061.2020135184350.psmcgscs_000501444664.Optical_Depth_Land_An

downloading:  raw/order/MOD04_3K.A2017333.1725.061.2017334014355.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2017335.1715.061.2017336013707.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2017336.mosaic.061.2020135184639.psmcgscs_000501444664.Optical_Depth_Land_And_Ocean.hdf
downloading:  raw/order/MOD04_3K.A2017337.mosaic.061.2020135184224.psmcgscs_000501444664.Optical_Depth_Land_And_Ocean.hdf
downloading:  raw/order/MOD04_3K.A2017338.1745.061.2017339014822.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2017339.1650.061.2017340014514.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2017340.1735.061.2017341015211.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2017341.mosaic.061.2020135184951.psmcgscs_000501444664.Optical_Depth_Land_And_Ocean.hdf
downloading:  raw/order/MOD04_3K.A2017342.1720.061.2017346193333.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2017343.1805.061.2017346194155.psgscs_000501444664.hdf
downloading

HTTP GET error code: 504
HTTP GET error message: 


downloading:  raw/order/MOD04_3K.A2018008.mosaic.061.2020135184441.psmcgscs_000501444664.Optical_Depth_Land_And_Ocean.hdf
downloading:  raw/order/MOD04_3K.A2018009.1720.061.2018010014508.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2018010.1805.061.2018011073158.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2018011.1710.061.2018012014126.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2018012.1750.061.2018013015317.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2018013.1655.061.2018014014943.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2018014.1740.061.2018015015040.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2018015.mosaic.061.2020135184318.psmcgscs_000501444664.Optical_Depth_Land_And_Ocean.hdf
downloading:  raw/order/MOD04_3K.A2018016.1725.061.2018017013945.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2018018.1715.061.2018019012017.psgscs_000501444664.hdf
downloading:  raw/order/MOD04_3K.A2018019.mo

HTTP GET error code: 504
HTTP GET error message: 


...

HTTP GET error code: 504
HTTP GET error message: 


downloading:  raw/order/README.1589482154_43308.7009.501444664.txt
downloading:  raw/order/README.1589482155_14357.7007.501444664.txt
downloading:  raw/order/README.1589482156_7293.7005.501444664.txt
downloading:  raw/order/README.1589482156_9573.7333.501444664.txt
downloading:  raw/order/README.1589482156_14360.7007.501444664.txt
downloading:  raw/order/README.1589482157_7296.7005.501444664.txt
downloading:  raw/order/README.1589482157_43305.7009.501444664.txt
downloading:  raw/order/README.1589482158_9572.7333.501444664.txt
downloading:  raw/order/README.1589482158_43306.7009.501444664.txt
downloading:  raw/order/README.1589482159_14356.7007.501444664.txt
downloading:  raw/order/README.1589482159_30604.7006.501444664.txt
downloading:  raw/order/README.1589482160_7299.7005.501444664.txt
downloading:  raw/order/README.1589482160_30600.7006.501444664.txt
downloading:  raw/order/README.1589482162_7294.7005.501444664.txt
downloading:  raw/order/README.1589482162_43307.7009.501444664.txt
d

HTTP GET error code: 504
HTTP GET error message: 


downloading:  raw/order/SUBSET_NOTE.3.725983818.3527_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725983819.29427_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725983882.4994_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725983885.3767_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725983891.7629_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725983952.43875_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725983986.29143_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725984000.46495_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725984059.20386_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725984079.1195_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725984094.5815_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725984112.44722_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725984156.27018_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725984168.47325_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725984181.29844_2.warn
downloading:  raw/order/SUBSET_NOTE.3.725984210.29876_2.warn
downloading:  raw/order/SUBSET

0

***